# Lese web_logs som Canvas_history
Først importerer eg alle nødvendige funksjonar:

In [1]:
import akvut
import pandas as pd

Så les eg `web_logs` frå det siste døgeret (det tok 6'47.2''):

In [2]:

logger = akvut.lag_logger('timer.log')
data = akvut.les_web_logs(logger)
print(data.info())


2025-02-12 18:17:43,623 - my_logger - INFO - Henta access_token OK.
2025-02-12 18:17:44,080 - my_logger - INFO - payload er {"format": "csv", "since": "2025-02-11T18:17:44.080386Z"}
2025-02-12 18:17:44,084 - my_logger - INFO - Sender (inkrementel) spørjing til https://api-gateway.instructure.com/dap/query/canvas_logs/table/web_logs/data


Har ikkje henta frå Azure


2025-02-12 18:17:45,113 - my_logger - INFO - Sjekker status på jobb 7103210b-fc2d-4093-8223-0c47b3fb1c0b
2025-02-12 18:17:50,849 - my_logger - INFO - Sjekker status på jobb 7103210b-fc2d-4093-8223-0c47b3fb1c0b
2025-02-12 18:17:56,462 - my_logger - INFO - Sjekker status på jobb 7103210b-fc2d-4093-8223-0c47b3fb1c0b
2025-02-12 18:18:02,073 - my_logger - INFO - Sjekker status på jobb 7103210b-fc2d-4093-8223-0c47b3fb1c0b
2025-02-12 18:18:07,800 - my_logger - INFO - Sjekker status på jobb 7103210b-fc2d-4093-8223-0c47b3fb1c0b
2025-02-12 18:18:13,464 - my_logger - INFO - Sjekker status på jobb 7103210b-fc2d-4093-8223-0c47b3fb1c0b
2025-02-12 18:18:19,031 - my_logger - INFO - Sjekker status på jobb 7103210b-fc2d-4093-8223-0c47b3fb1c0b
2025-02-12 18:18:24,647 - my_logger - INFO - Sjekker status på jobb 7103210b-fc2d-4093-8223-0c47b3fb1c0b
2025-02-12 18:18:30,149 - my_logger - INFO - Sjekker status på jobb 7103210b-fc2d-4093-8223-0c47b3fb1c0b
2025-02-12 18:18:35,932 - my_logger - INFO - Sjekker st

Feil ved opplasting av sist oppdatert: ('42000', "[42000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Cannot open server 'hvl-data-db-server' requested by the login. Client with IP address '46.212.54.43' is not allowed to access the server.  To enable access, use the Azure Management Portal or run sp_set_firewall_rule on the master database to create a firewall rule for this IP address or address range.  It may take up to five minutes for this change to take effect. (40615) (SQLDriverConnect)")
<class 'pandas.core.frame.DataFrame'>
Index: 2489080 entries, 0 to 1379042
Data columns (total 25 columns):
 #   Column                              Dtype 
---  ------                              ----- 
 0   key.id                              object
 1   value.timestamp                     object
 2   value.user_id                       object
 3   value.real_user_id                  object
 4   value.course_id                     object
 5   value.quiz_id                       objec

Når eg les `web_logs` lagrer eg rådata i ei fil (`ny_history_alle.csv`). Men eg kan no sende desse vidare til nye rutiner der eg henter ut andre typer data. 

Litt usikker på om det er rett utval eg gjer her.

In [3]:

akvut.ny_history(data, logger)

2025-02-12 18:23:25,871 - my_logger - INFO - Skriv 2489080 linjer i ny_history_alle.csv (som test)
2025-02-12 18:23:59,737 - my_logger - INFO - Antall linjer før reduksjon: 2489080
2025-02-12 18:24:00,017 - my_logger - INFO - Fjerner linjer med api
2025-02-12 18:24:01,118 - my_logger - INFO - Fjerner linjer med retrieve
2025-02-12 18:24:03,842 - my_logger - INFO - Antall linjer etter reduksjon: 401480


## Alias
Eg prøver å lage statistikk over kven som opptrer som kven. Her har eg `user_id` på dei mest vanlege superbrukarane, og så kan eg lage ei liste over 

- kven dei har opptrådd som
- kor mange ganger dei har opptredd

NB! Det er mange av desse som er "Studentvising", slik at det er mange "user_id" som ikkje finst.


In [4]:
alias = data[~data['value.real_user_id'].isnull()]
adm_namn = {'2916': 'Aasmund', '12353': 'Johan', '3861': 'Casper', '3990': 'Dag Ove', '26737': 'Hege', '97393': 'Henrik', '35965': 'Jan Erik', '14186': 'Andreas Senneset Hove', '6': 'Ivar', '1703': 'Terje'}
adm_liste = []
for adm in ["1703", "2916", "12353", "97393", "3861", "14186", "35965", "98283", "5036", "6", "86785", "26737", "3990"]:
    adm_liste.append({'user': adm, 'count': alias[alias['value.real_user_id'] == adm]['value.user_id'].value_counts()})
for a in adm_liste:
    if len(a['count']) > 0:
        print(f"Administrator: {a['user']} ({adm_namn[a['user']]})")
        for index, item in a['count'].items():
            print(f"{index}: {item}")


Administrator: 2916 (Aasmund)
106420: 148
110226: 27
109587: 12
Administrator: 12353 (Johan)
103078: 23
Administrator: 3990 (Dag Ove)
4975: 13


## Data (til history)
Så til det eg eigentleg skulle gjere: hente ut data til U4-prosjektet. Det er fleire krav:
1. Feltet `user_id` må innehalde noko
2. Feltet `value.http_status` skal være `200`
3. Feltet `value.url` skal ikkje starte på "/api/"
4. Feltet `value.url` skal ikkje innehalde "preview"

4. I feltet `value.web_application_action` skal eg ta vekk alle som er:
 - blanke
 - "content_tag_assignment_data"
 - "progressions"
 - "backup"

In [21]:
temp = data[(data["value.user_id"].notnull()) & (data["value.http_status"].str.startswith("2")) & (~data["value.url"].str.startswith("/api/")) & (~data["value.url"].str.contains("preview"))]
utval = temp[(~temp['value.web_application_action'].isin(['content_tag_assignment_data', 'progressions', 'backup']))]

In [22]:
utval

,key.id,value.timestamp,value.user_id,value.real_user_id,value.course_id,value.quiz_id,value.discussion_id,value.conversation_id,value.assignment_id,value.url,...,value.web_application_controller,value.web_application_action,value.web_application_context_type,value.web_application_context_id,value.session_id,value.developer_key_id,value.participated,value.user_agent,meta.ts,meta.action
7,0cdd89b7-ecb9-41be-afa3-7fc772e1ccc8,2025-02-12T09:00:18.542Z,104999,NaN,NaN,NaN,NaN,NaN,NaN,/page_views/71d050cf-5858-40f0-9d9e-c2cf63c390...,...,page_views,update,NaN,NaN,a64792f6565ecca2ea6eba3ac77a8309,NaN,false,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2025-02-12T10:50:02.207Z,U
15,82e8e5a5-f514-4292-88b6-a4b9ad5af10c,2025-02-12T09:00:21.313Z,93224,NaN,NaN,NaN,NaN,NaN,NaN,/login/session_token?return_to=https://hvl.ins...,...,login,session_token,NaN,NaN,247a585b1534ca66b896b7c810cb3933,170000000000004,false,iCanvas/7.11.1 (24341) iPhone/iOS 18.3,2025-02-12T10:50:02.207Z,U
20,1bc3538f-563b-4bd1-97f4-245331e03fdd,2025-02-12T09:00:23.136Z,68816,NaN,NaN,NaN,NaN,NaN,NaN,/,...,users,user_dashboard,User,68816,7f8eaf1148d19ea37cb5b15090a371cd,NaN,false,Mozilla/5.0 (iPhone; CPU iPhone OS 18_1_1 like...,2025-02-12T10:50:02.207Z,U
45,39775f53-d9c4-4209-94d3-a8a1d9339b07,2025-02-12T09:00:31.285Z,101782,NaN,28886,NaN,NaN,NaN,NaN,/courses/28886/pages/12-dot-02-kasuistikk-muli...,...,wiki_pages,show,Course,28886,f8b3f8ea6be78cc9255bf7cdf880bd0d,NaN,false,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,2025-02-12T10:50:02.207Z,U
51,06b59aff-83cf-4ffd-8e93-e7859043acbd,2025-02-12T09:00:31.664Z,102693,NaN,NaN,NaN,NaN,NaN,NaN,/,...,users,user_dashboard,User,102693,64f1872e67b1366c83f2763bb5ca3ee6,NaN,false,Mozilla/5.0 (iPhone; CPU iPhone OS 17_6_1 like...,2025-02-12T10:50:02.207Z,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1378918,62e904c7-b2dc-4e6c-8ebc-40ef28f0e870,2025-02-12T03:03:11.213Z,46794,NaN,NaN,NaN,NaN,NaN,NaN,/feeds/calendars/user_0hnkyobKAnYybM1J0BKjPElC...,...,calendar_events_api,public_feed,User,46794,NaN,NaN,false,macOS/14.5 (23F79) dataaccessd/1.0,2025-02-12T03:47:58.863Z,U
1378927,a534524f-50bb-4063-8ec1-5f0321731d0b,2025-02-12T03:03:14.212Z,94085,NaN,28251,NaN,NaN,NaN,NaN,/courses/28251/files/3193460?module_item_id=90...,...,files,show,Course,28251,9945ae6d3af681f1059414ddc71b834e,NaN,false,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2025-02-12T03:47:58.863Z,U
1378950,f691f1eb-f454-4129-8a00-88709b4bdbe1,2025-02-12T03:03:25.023Z,104035,NaN,NaN,NaN,NaN,NaN,NaN,/feeds/calendars/user_fxjQse2l281IGDxk7Sn3kPob...,...,calendar_events_api,public_feed,User,104035,NaN,NaN,false,iOS/18.3 (22D63) dataaccessd/1.0,2025-02-12T03:47:58.863Z,U
1378975,0895cb44-de74-4b9d-8c15-d89336809a7c,2025-02-12T03:03:52.225Z,51252,NaN,NaN,NaN,NaN,NaN,NaN,/feeds/calendars/user_HcHfCo6VQxwvWhlxLo62skcl...,...,calendar_events_api,public_feed,User,51252,NaN,NaN,false,iOS/18.3 (22D63) dataaccessd/1.0,2025-02-12T03:47:58.863Z,U


In [25]:
utval['value.web_application_action'].value_counts()

value.web_application_action
session_token                          65000
show                                   51248
user_dashboard                         31259
index                                  21709
web_app_manifest                       20682
                                       ...  
unpublish                                  1
publish_quizzes                            1
toggle_hide_dashcard_color_overlays        1
prior_users                                1
roster_user_services                       1
Name: count, Length: 86, dtype: int64

Det er litt usikkert kva vi skal bruke det til, men eg kan sende data til Håvard og be han plukke ut. 

In [ ]:
utval.to_csv("history_alle.csv")

Eg kan kanskje sende berre dei som er interessante (håper eg treff på utvalet):

In [26]:
redusert_utval = utval[['value.timestamp', 'value.user_id', 'value.course_id', 'value.url', 'value.web_application_controller', 'value.web_application_action', 'value.web_application_context_type']]


In [27]:
redusert_utval['dato'] = redusert_utval['value.timestamp'].apply(lambda x: x[0:10])

/tmp/ipykernel_3474/2458858629.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  redusert_utval['dato'] = redusert_utval['value.timestamp'].apply(lambda x: x[0:10])


In [28]:
redusert_utval.to_csv("history_redusert_utval.csv")

Her er ei linje for å hente data inn att frå fil:

In [ ]:
redusert_utval = pd.read_csv("history_250210_redusert_utval.csv")

Eg kan lage ei liste over kor mange aktive hendingar det er per student:

In [29]:
redusert_utval['value.user_id'].value_counts()

value.user_id
105833    410
12353     392
110265    346
101531    308
105994    303
         ... 
73145       1
17294       1
82887       1
56594       1
78369       1
Name: count, Length: 10868, dtype: int64

In [35]:
redusert_utval

,value.timestamp,value.user_id,value.course_id,value.url,value.web_application_controller,value.web_application_action,value.web_application_context_type,dato
7,2025-02-12T09:00:18.542Z,104999,NaN,/page_views/71d050cf-5858-40f0-9d9e-c2cf63c390...,page_views,update,NaN,2025-02-12
15,2025-02-12T09:00:21.313Z,93224,NaN,/login/session_token?return_to=https://hvl.ins...,login,session_token,NaN,2025-02-12
20,2025-02-12T09:00:23.136Z,68816,NaN,/,users,user_dashboard,User,2025-02-12
45,2025-02-12T09:00:31.285Z,101782,28886,/courses/28886/pages/12-dot-02-kasuistikk-muli...,wiki_pages,show,Course,2025-02-12
51,2025-02-12T09:00:31.664Z,102693,NaN,/,users,user_dashboard,User,2025-02-12
...,...,...,...,...,...,...,...,...
1378918,2025-02-12T03:03:11.213Z,46794,NaN,/feeds/calendars/user_0hnkyobKAnYybM1J0BKjPElC...,calendar_events_api,public_feed,User,2025-02-12
1378927,2025-02-12T03:03:14.212Z,94085,28251,/courses/28251/files/3193460?module_item_id=90...,files,show,Course,2025-02-12
1378950,2025-02-12T03:03:25.023Z,104035,NaN,/feeds/calendars/user_fxjQse2l281IGDxk7Sn3kPob...,calendar_events_api,public_feed,User,2025-02-12
1378975,2025-02-12T03:03:52.225Z,51252,NaN,/feeds/calendars/user_HcHfCo6VQxwvWhlxLo62skcl...,calendar_events_api,public_feed,User,2025-02-12


## Leit etter duplikat
Det neste vert å leite etter duplikat. Det vil sei: rader som har lik verdi i fleire celler. Då må eg eigentleg lage ei ny dataramme; eg går gjennom den første og finn like rader, og så legg eg denne raden (og antalet) i ei ny ramme. Vi prøver med noko enkelt:

`user_id`, `course_id`, `web_application_controller`, `web_application_action` og `dato` er like.


In [30]:
temp_duplikat = redusert_utval[["value.user_id", "value.course_id", "value.web_application_controller", "value.web_application_action", "value.web_application_context_type", "dato"]].value_counts()
duplikat = temp_duplikat.to_frame('frekvens').reset_index()

Og så lagrer eg til fil før eg ser på detaljane:

In [31]:
duplikat.to_csv('duplikat.csv')

Først er eg på dei fem mest brukte linjene:

In [33]:
duplikat.head()

,value.user_id,value.course_id,value.web_application_controller,value.web_application_action,value.web_application_context_type,dato,frekvens
0,105833,30023,external_tools,retrieve,Course,2025-02-12,189
1,8620,30829,external_tools,retrieve,Course,2025-02-12,160
2,105833,30023,external_tools,retrieve,Course,2025-02-11,153
3,106119,30023,external_tools,retrieve,Course,2025-02-12,112
4,93417,30297,external_tools,retrieve,Course,2025-02-12,94


Her har altså student 106979 gjort same kommando 265 ganger! For å finne ut kva det er må eg nesten sjekke med rådata (av di dette er ein quiz har eg ikkje tatt med data om quizen i utvalet, og eg må drive litt detektivarbeid for å finne kva quiz det er):

In [ ]:
student_106979 = data[(data['value.user_id'] == "106979") & (data['value.quiz_id'] == "18705")]
student_106979.to_csv('student_106979.csv', index=False)

Så tar eg ut ei liste over kor mange ganger kva duplikat-rad opptrer; det viser seg jo at dei aller, aller fleste radene er unike:

In [34]:
duplikat['frekvens'].value_counts().sort_index()

frekvens
1      22459
2       7876
3       3420
4       1837
5        967
       ...  
94         1
112        1
153        1
160        1
189        1
Name: count, Length: 62, dtype: int64

## Antal unike wiki_pages besøkt
Vi kan også sjå på kor mange ganger kvar enkelt wiki_side er besøkt:

In [36]:
temp99 = redusert_utval[redusert_utval["value.web_application_controller"]=="wiki_pages"]
håvard = temp99[["value.user_id", "value.course_id", "value.url"]].value_counts()
håvard_ramme = håvard.to_frame('frekvens').reset_index()

In [37]:
håvard_ramme.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5115 entries, 0 to 5114
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   value.user_id    5115 non-null   object
 1   value.course_id  5115 non-null   object
 2   value.url        5115 non-null   object
 3   frekvens         5115 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 160.0+ KB


In [38]:
håvard_ramme

,value.user_id,value.course_id,value.url,frekvens
0,1703,86,/courses/86/pages/alt-1-guide,22
1,52137,30810,/courses/30810/pages/pensum-og-forberedelsesma...,20
2,52137,30810,/courses/30810/pages/pensum-og-forberedelsesma...,19
3,47484,28705,/courses/28705/pages,16
4,105890,28869,/courses/28869/pages/outline-enu804n-v2025,12
...,...,...,...,...
5110,98999,28283,/courses/28283/pages/tidligere-eksamensoppgaver,1
5111,98999,28875,/courses/28875/pages/tirsdag-28-dot-1-f05-stab...,1
5112,99332,29235,/courses/29235/pages/eksamen-makp402?module_it...,1
5113,98715,29164,/courses/29164/pages/eldre-forelesningsvideoer,1
